# 随机森林RF

理解了bagging算法，随机森林(Random Forest,以下简称RF)就好理解了。

它是Bagging算法的进化版，也就是说，它的思想仍然是bagging，但是进行了独有的改进。

我们现在就来看看RF算法改进了什么。　　　

**首先**，RF使用了CART决策树作为弱学习器。

**第二**，在使用决策树的基础上，RF对决策树的建立做了改进，对于普通的决策树，我们会在节点上所有的n个样本特征中选择一个最优的特征来做决策树的左右子树划分，但是**RF通过随机选择节点上的一部分样本特征**，这个数字小于n，假设为$n_{sub}$，然后在这些随机选择的$n_{sub}$个样本特征中，选择一个最优的特征来做决策树的左右子树划分。这样进一步增强了模型的泛化能力。

如果$n_{sub}=n$，则此时RF的CART决策树和普通的CART决策树没有区别。

$n_{sub}$越小，则模型越健壮，当然此时对于训练集的拟合程度会变差。也就是说$n_{sub}$越小，模型的方差会减小，但是偏倚会增大。

在实际案例中，一般会通过交叉验证调参获取一个合适的$n_{sub}$的值。


除了上面两点，RF和普通的bagging算法没有什么不同， 下面简单总结下RF的算法。


```
输入为样本集$D={(x_1,y_1),(x_2,y_2),...(x_m,y_m)}$，弱分类器迭代次数T。输出为最终的强分类器$f(x)$:
```

1）对于$t=1,2...,T$：

 - a)对训练集进行第t次随机采样，得到一个包含m个样本的采样集$D_m$
 - b)用采样集$D_m$训练第m个决策树模型$G_m(x)$，在训练决策树模型的节点的时候， 在所有的特征中选择一部分特征，在这些随机选择的部分样本特征中选择最优的特征来做决策树的左右子树划分。

2) 如果是分类算法预测，则T个弱学习器投出最多票数的类别或者类别之一为最终类别。如果是回归算法，T个弱学习器得到的回归结果进行算术平均得到的值为最终的模型输出。

作为一个可以高度并行化的算法，RF在大数据时候大有可为。这里也对常规的随机森林算法的优缺点做一个总结。

**RF的主要优点有：**

1）训练可以高度并行化，对于大数据时代的大样本训练速度有优势。个人觉得这是的最主要的优点。

2）由于可以随机选择决策树节点划分特征，这样在样本特征维度很高的时候，仍然能高效的训练模型。

3）在训练后，可以给出各个特征对于输出的重要性

4）由于采用了随机采样，训练出的模型的方差小，泛化能力强。

5）相对于Boosting系列的Adaboost和GBDT， RF实现比较简单。

6）对部分特征缺失不敏感。

随机森林的准确率可以和adaboost相媲美，但是对错误和离群点更鲁棒。

随机森林中树的个数增加，森林的泛化误差收敛，因此过拟合不是问题。

随机森林的准确率依赖于个体分类器的实力和他们之间的依赖性。理想情况是保持个体分类的能力而不提高他们之间的相关性。

随机森林对每次划分所考虑的属性数目很敏感。通常选取$log_2^d+1$个属性。

**RF的主要缺点有：**

1)在某些噪音比较大的样本集上，RF模型容易陷入过拟合。

2)取值划分比较多的特征容易对RF的决策产生更大的影响，从而影响拟合的模型的效果。

# AdaBoost

AdaBoost是由Freund与Schapire提出来解决二分类问题$y∈\{1,-1\} $，其定义损失函数为指数损失函数：

$$L(y,f(x))=exp(−yf(x)) $$

则整个样本集(n个样本)的整体损失函数，或者叫目标函数为

$$L(y,f(x))=\frac{1}{n}\sum_{i=1}^nexp(−y_if(x_i)) $$

我们来看看是如何根据损失函数推导出模型系数$α_m$的。

由于多个基分类器总是依次出现，所以我们不会当所有分类器出现以后再计算每个分类器的权重，而是每次出现一个基分类器，我们就根据损失函数，计算当前分类器的权重。

（每个分类器的权重还会影响下一个分类器的样本权重，所以必然是每训练出一个分类器，就计算一个分类器的权重）。

关于为什么要使用指数损失，可以参考：https://www.cnblogs.com/willnote/p/6801496.html 或者 https://blog.csdn.net/thriving_fcl/article/details/50877957


根据加型模型（additive model），第m轮的分类函数

$$f_m(x)=f_{m−1}(x)+α_m G_m (x) $$

其中，$α_m$为基分类器$G_m(x)$的组合系数。AdaBoost采用前向分布（forward stagewise）这种贪心算法最小化损失函数，求解子模型的$α_m$

$$α_m =\frac{1}{2} log\frac{1−e_m}{e_m}$$

其中，$e_m$为$G_m(x)$的加权分类误差率
$$e_m = \frac {\sum_{i=1}^N w_i^{(m)} I(y_i \ne G(x_i))} {\sum_{i=1}^N w_i^{(m)}}$$

第$m+1$轮的训练数据集权值分布
$D _{m+1} =(w_{m+1},1 ,⋯,w_{m+1},i ,⋯,w_{m+1},N )$

$$w_{m+1,i} =\frac{w_{m,i}}{Z_m}  exp(−α_my_iG_m (x_i)) $$

其中，$Z_m$为规范化因子

$$Z_m =\sum_{i=1}^N w_{m,i} ∗exp(−α_m y_i G_m (x_i )) $$

则得到最终分类器

$$sign(f(x))=sign(\sum_{m=1}^M α_m G_m (x)) $$

$α_m$是$e_m$的单调递减函数，特别地，当$e_m ≤0.5$时，$α_m$ ≥0；当$e_m >12 $时，即基分类器不满足弱可学习的条件（比随机猜测好），则应该停止迭代。

具体算法流程如下：

$D_1 (i)=1/N$ %初始化权重分布

 for m=1,⋯,M m=1,⋯,M:
 
 　　learn base classifier $G_m(x)$;
   
　　 if $e_m>0.5$ then break；
   
　　update $α_m$and $D_{m+1}$;
  
end for

在算法第4步，学习过程有可能停止，导致学习不充分而泛化能力较差。

因此，可采用“重采样”（re-sampling）避免训练过程过早停止；即抛弃当前不满足条件的基分类器，基于重新采样的数据训练分类器，从而获得学习“重启动”机会。

AdaBoost能够自适应（addaptive）地调整样本的权值分布，将分错的样本的权重设高、分对的样本的权重设低；所以被称为“Adaptive Boosting”。

sklearn的AdaBoostClassifier实现了AdaBoost，默认的基分类器是能fit()带权值样本的DecisionTreeClassifier。

下面是$α_m$和$G_m (x) $的推导过程，不喜欢的可以直接跳过去

----------
损失函数已知为下面的公式
$$L(y,f(x))=\frac{1}{n}\sum_{i=1}^nexp(−y_if(x_i)) $$

我们的目标是求使得到的$α_m$和$G_m(x)$令$L(y,f(x))$最小，即：

$$(\alpha_{m},G_{m}(x))=\underset{\alpha_{m},G_{m}}{arg\ min}\sum_{i=1}^{n}e^{-y_{i}(f_{m-1}(x)+\alpha_{m}G_{m}(x))}\\
=\underset{\alpha_{m},G_{m}}{arg\ min}\sum_{i=1}^{n}\bar{w}_{mi}e^{-y_{i}\alpha_{m}G_{m}(x)}\\
G_{m}(x)^{*}=\underset{G_{m}(x)}{arg\ min}\sum_{i=1}^{n}\bar{w}_{mi}I(y_{i}\neq G_{m}(x))$$

其中，$\bar{w}_{mi}=e^{-y_{i}f_{m-1}(x)}$。因为$\bar{w}_{mi}$既不依赖于$α_m$也不依赖于$G_m(x)$，所以与最小化无关。但它依赖于$f_{m−1}(x)$，会随着每一轮迭代而发生变化。第二个式子为令指数损失函数最小的$G_m(x)∗$，其中$I(⋅)$为指示函数，此$G_m(x)∗$使第$m$轮加权训练数据分类的误差率得到了最小值。接下来我们对上边的第一个式子的右边进行一下简单变形：

$$\sum_{i=1}^{n}\bar{w}_{mi}e^{y_{i}\alpha_{t}G_{t}(x_{i})}\\
=\sum_{y_{i}=G_{t}(x_{i})}\bar{w}_{mi}e^{-\alpha_{m}}+\sum_{y_{i}\neq G_{m}(x_{i})}\bar{w}_{mi}e^{\alpha_{m}}\\
=(e^{\alpha_{m}}-e^{-\alpha_{m}})\sum_{i=1}^{n}\bar{w}_{mi}I(y_{i}\neq G_{m}(x_{i}))+e^{-\alpha_m}\sum_{i=1}^{n}\bar{w}_{mi}$$

将上式对$α_m$求导并令导数为0，即可解得：
$$\alpha_{m}^{*}=\frac{1}{2}log\frac{1-e_{m}}{e_{m}}$$

上式即之前例子中所用到的$α_m$的更新公式，其中，$e_m$为分类误差率：

$$e_{m}=\frac{\sum_{i=1}^{n}\bar{w}_{mi}I(y_{i}\neq G_{m}(x_{i}))}{\sum_{i=1}^{n}\bar{w}_{mi}}\\
=\sum_{i=1}^{n}w_{mi}I(y_{i}\neq G_{m}(x_{i}))$$


----------

**算法案例代码实现：**

https://github.com/626626cdllp/data-mining/tree/master/ensemble-learning/adaboost